In [1]:
import numpy as np
from scipy.stats import norm
import scipy.integrate as integrate

In [2]:
def BlackScholesCall(S0, K, r, sigma, T):
    d1 = (np.log(S0/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    return S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

In [3]:
def BlackScholesPut(S0, K, r, sigma, T):
    d1 = (np.log(S0/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    return K*np.exp(-r*T)*norm.cdf(-d2) - S0*norm.cdf(-d1)

In [4]:
def impliedvol(K):
    return 0.51 - 0.591*K + 0.376*K**2 - 0.105*K**3 + 0.011*K**4

In [5]:
def capped_impliedvol(K):
    if K < 3:
        return impliedvol(K)
    else:
        return impliedvol(3)

In [6]:
# driver routine
r = 0.00; 
q = 0.00; 
T = 4; 
S0 = 1; 
DF = np.exp(-r*T)

In [7]:
def numerical_integration_sqrt(S0, r, q, T, SD):
    
    DF = np.exp(-r*T)
    DivF = np.exp(-q*T)
    f = S0*DivF/DF
    
    # Implied vol at the strike
    Implied_vol = capped_impliedvol(f)
    
    # Upper limit of the call integral:SD-number of standard deviation away from mean
    maxS = f*np.exp(Implied_vol * SD * np.sqrt(T))

    # Discounted payoff
    forward_part = np.sqrt(f) * np.exp(-r*T)
    
    integrated_put = lambda p: p**(-1.5)/4 * BlackScholesPut(S0, p, r, capped_impliedvol(p), T)
    put_part, error = integrate.quad(integrated_put, 1e-4, f)
    
    integrated_call = lambda c: c**(-1.5)/4 * BlackScholesCall(S0, c, r, capped_impliedvol(c), T)
    call_part, error = integrate.quad(integrated_call, f, maxS)

    return forward_part - put_part - call_part

In [8]:
def numerical_integration_cube(S0, r, q, T, SD):
    
    DF = np.exp(-r*T)
    DivF = np.exp(-q*T)
    f = S0*DivF/DF
    
    # Implied vol at the strike
    Implied_vol = capped_impliedvol(f)
    
    # Upper limit of the call integral:SD-number of standard deviation away from mean
    maxS = f*np.exp(Implied_vol * SD * np.sqrt(T))

    # Discounted payoff
    forward_part = f**3 * np.exp(-r*T)
    
    integrated_put = lambda p: 6*BlackScholesPut(S0, p, r, capped_impliedvol(p), T)*p
    put_part, error = integrate.quad(integrated_put, 1e-4, f)
    
    integrated_call = lambda c: 6*BlackScholesCall(S0, c, r, capped_impliedvol(c), T)*c
    call_part, error = integrate.quad(integrated_call, f, maxS)

    return forward_part + put_part + call_part

In [9]:
kappas = np.linspace(1,6,6)

In [10]:
kappas

array([1., 2., 3., 4., 5., 6.])

In [11]:
numIntSqrtResults = [numerical_integration_sqrt(S0, r, q, T, SD) for SD in kappas]

In [12]:
numIntSqrtResults

[0.9744529379057193,
 0.9737922020460309,
 0.9737631318280784,
 0.9737624998879959,
 0.9737624966184146,
 0.9737624965892451]

In [13]:
numIntCubeResults = [numerical_integration_cube(S0, r, q, T, SD) for SD in kappas]

In [14]:
numIntCubeResults

[1.4563473950073509,
 1.5157020030493475,
 1.5226686291504619,
 1.5230535930212965,
 1.5230590092868386,
 1.5230590334482457]